#### 환경

conda create -n qwen3 python=3.10   
conda activate qwen3   
pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124    ## pytorch    
pip install transformers accelerate bitsandbytes sentencepiece protobuf   ## LLM 모델    
pip install huggingface_hub     # 허깅페이스 로그인    

#### 모델 선택

- 모델 파일크기로 필요 GPU메모리량 확인
- 예) RTX 3090*2개 Qwen3-14B 모델 활용
- huggingface 에 token 있으면 빠른 다운로드

In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gc
import os

class Qwen3_14B:
    def __init__(self, hf_token=None):
        self.model_name = "Qwen/Qwen3-14B"
        
        print("="*70)
        print("Qwen3-14B Loader")
        print("="*70)
        print()
        print(f"Model: {self.model_name}")
        print(f"Download size: 29.6GB")
        print(f"Expected memory: ~38GB during inference")
        print(f"Available VRAM: 48GB")
        print()
        
        # GPU 확인
        num_gpus = torch.cuda.device_count()
        print(f"GPUs available: {num_gpus}")
        for i in range(num_gpus):
            props = torch.cuda.get_device_properties(i)
            print(f"  GPU {i}: {props.name} - {props.total_memory / 1024**3:.1f}GB")
        print()
        
        # 메모리 정리
        print("Cleaning GPU memory...")
        torch.cuda.empty_cache()
        gc.collect()
        print()
        
        # Tokenizer
        print("Step 1/2: Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            token=hf_token,
            trust_remote_code=True
        )
        print("  Tokenizer loaded")
        print()
        
        # Model
        print("Step 2/2: Loading model (this takes 3-5 minutes)...")
        print("  Downloading 3 shards: 9.95GB + 9.89GB + 9.72GB")
        print()
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            token=hf_token,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        
        print()
        print("Model loaded successfully!")
        print()
        
        self._print_memory()
    
    def _print_memory(self):
        """메모리 사용량 상세 출력"""
        print("="*70)
        print("GPU Memory Usage")
        print("="*70)
        print()
        
        total_allocated = 0
        total_reserved = 0
        
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            total_mem = torch.cuda.get_device_properties(i).total_memory / 1024**3
            
            total_allocated += allocated
            total_reserved += reserved
            
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"  Allocated:  {allocated:>6.2f}GB / {total_mem:.1f}GB ({allocated/total_mem*100:>5.1f}%)")
            print(f"  Reserved:   {reserved:>6.2f}GB")
            print(f"  Free:       {total_mem - reserved:>6.2f}GB")
            print()
        
        total_vram = torch.cuda.device_count() * 24
        
        print(f"Total:")
        print(f"  Allocated:  {total_allocated:>6.2f}GB / {total_vram}GB ({total_allocated/total_vram*100:>5.1f}%)")
        print(f"  Available:  {total_vram - total_allocated:>6.2f}GB")
        
        # 레이어 분산
        if hasattr(self.model, 'hf_device_map'):
            print()
            print("Layer distribution:")
            device_layers = {}
            for name, device in self.model.hf_device_map.items():
                device_str = str(device)
                device_layers[device_str] = device_layers.get(device_str, 0) + 1
            
            for device, count in sorted(device_layers.items()):
                print(f"  {device}: {count} layers")
        
        print()
        print("="*70)
        print()
    
    def chat(self, message, max_tokens=2000, temperature=0.7):
        """채팅"""
        messages = [
            {"role": "system", "content": "당신은 ACMG/AMP 변이 해석분야의 임상 유전학 전문가입니다."},
            {"role": "user", "content": message}
        ]
        
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        
        print("Generating response...")
        
        import time
        start = time.time()
        
        with torch.no_grad():
            # 메모리 정리
            torch.cuda.empty_cache()
            
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        elapsed = time.time() - start
        num_tokens = len(outputs[0]) - len(inputs[0])
        
        response = self.tokenizer.decode(
            outputs[0][len(inputs[0]):],
            skip_special_tokens=True
        )
        
        print()
        print("="*70)
        print(f"Generated {num_tokens} tokens in {elapsed:.2f}s ({num_tokens/elapsed:.1f} tok/s)")
        print("="*70)
        print()
        print(response)
        print()
        
        return response
    
 
    
    def analyze_variant(self, variant_info):
        """ACMG/AMP 2015 변이 분석"""
        prompt = f"""당신은 ACMG/AMP 2015 가이드라인과 ClinGen Sequence Variant Interpretation (SVI) 권고안에 따라 변이 해석을 수행하는 임상 유전학 전문가입니다.

변이 정보:
유전자: {variant_info.get('gene')}
변이: {variant_info.get('variant')}
유형: {variant_info.get('type')}
cDNA: {variant_info.get('cdna', 'N/A')}
단백질: {variant_info.get('protein', 'N/A')}

증거 데이터:
집단 데이터:
- gnomAD AF (전체): {variant_info.get('gnomad_af', 'Unknown')}
- gnomAD AF (popmax): {variant_info.get('gnomad_popmax', 'Unknown')}
- gnomAD 동형접합: {variant_info.get('gnomad_hom', 'Unknown')}

데이터베이스 증거:
- ClinVar: {variant_info.get('clinvar', 'Not found')}
- HGMD: {variant_info.get('hgmd', 'Not available')}

전산 예측:
- CADD: {variant_info.get('cadd', 'N/A')}
- REVEL: {variant_info.get('revel', 'N/A')}
- SpliceAI: {variant_info.get('spliceai', 'N/A')}
- 보존성 (phyloP): {variant_info.get('phylop', 'N/A')}

기능 연구:
- In vitro 데이터: {variant_info.get('functional', 'N/A')}
- In vivo 데이터: {variant_info.get('in_vivo', 'N/A')}

가계 분리 및 증례 데이터:
- 가계 분리: {variant_info.get('segregation', 'N/A')}
- De novo: {variant_info.get('de_novo', 'N/A')}
- 환자-대조군 연구: {variant_info.get('case_control', 'N/A')}

과제: ACMG/AMP 2015 기준 체계적 평가

1. 병원성 증거:

   A. 매우 강력 (PVS1):
      - PVS1: LOF가 알려진 병인 기전인 유전자에서의 null 변이 (nonsense, frameshift, canonical ±1 or 2 splice sites, 개시코돈, 단일/다중 엑손 결실)
      → 평가: [충족/미충족]
      → 근거: [상세한 근거]
      → 강도: [PVS1, PVS1_Strong, PVS1_Moderate, PVS1_Supporting, 또는 미충족]

   B. 강력 (PS1-PS4):
      - PS1: 확립된 병원성 변이와 동일한 아미노산 변화
      → 평가: [충족/미충족]
      → 근거:
      
      - PS2: 질환이 있고 가족력이 없는 환자에서의 De novo (모계/부계 확인됨)
      → 평가: [충족/미충족]
      → 근거:
      
      - PS3: 잘 확립된 기능 연구에서 손상 효과 입증 (ClinGen SVI 기준)
      → 평가: [충족/미충족]
      → 근거: [고려사항: 검증된 분석법, 재현성, 기능적 효과 입증]
      → 강도: [PS3, PS3_Moderate, PS3_Supporting, 또는 미충족]
      
      - PS4: 환자군에서의 유병률이 대조군 대비 유의하게 증가
      → 평가: [충족/미충족]
      → 근거:

   C. 중등도 (PM1-PM6):
      - PM1: 양성 변이 없이 돌연변이 핫스팟 및/또는 중요 기능 도메인에 위치
      → 평가: [충족/미충족]
      → 근거:
      
      - PM2: 집단 데이터베이스(gnomAD)에서 부재 또는 극히 드묾
      → 평가: [충족/미충족]
      → 근거: [질환 유병률 및 발현율 고려]
      → 강도: [PM2, PM2_Supporting, 또는 미충족]
      
      - PM3: 열성 질환에서 병원성 변이와 trans에서 검출
      → 평가: [충족/미충족]
      → 근거:
      
      - PM4: 단백질 길이 변화 (in-frame indel, stop-loss)
      → 평가: [충족/미충족]
      → 근거:
      
      - PM5: 다른 병원성 미스센스가 존재하는 아미노산 위치의 새로운 미스센스
      → 평가: [충족/미충족]
      → 근거:
      
      - PM6: De novo로 가정 (모계/부계 미확인)
      → 평가: [충족/미충족]
      → 근거:

   D. 지지 (PP1-PP5):
      - PP1: 다수의 환자 가족 구성원에서 질환과 공동분리
      → 평가: [충족/미충족]
      → 근거:
      
      - PP2: 미스센스 변이가 적고 미스센스가 흔한 병인 기전인 유전자의 미스센스
      → 평가: [충족/미충족]
      → 근거:
      
      - PP3: 다수의 전산 증거가 유해 효과 지지
      → 평가: [충족/미충족]
      → 근거: [REVEL, BayesDel 같은 메타 예측자 고려]
      
      - PP4: 환자 표현형/가족력이 유전자에 매우 특이적
      → 평가: [충족/미충족]
      → 근거:
      
      - PP5: 신뢰할 수 있는 출처에서 병원성으로 분류
      → 평가: [충족/미충족]
      → 근거:

2. 양성 증거:

   A. 단독 (BA1):
      - BA1: 집단 데이터베이스에서 대립유전자 빈도 >5%
      → 평가: [충족/미충족]
      → 근거:

   B. 강력 (BS1-BS4):
      - BS1: 질환에 예상되는 것보다 높은 대립유전자 빈도
      → 평가: [충족/미충족]
      → 근거: [질환 유병률, 발현율, 유전적 이질성 고려]
      
      - BS2: 열성/우성 질환에 대해 건강한 성인에서 관찰
      → 평가: [충족/미충족]
      → 근거:
      
      - BS3: 잘 확립된 기능 연구에서 손상 효과 없음 (ClinGen SVI 기준)
      → 평가: [충족/미충족]
      → 근거: [고려사항: 검증된 분석법, 재현성, 기능적 효과 없음]
      → 강도: [BS3, BS3_Moderate, BS3_Supporting, 또는 미충족]
      
      - BS4: 환자 구성원에서 분리 부족
      → 평가: [충족/미충족]
      → 근거:

   C. 지지 (BP1-BP7):
      - BP1: 주로 절단 변이가 질환을 유발하는 유전자의 미스센스
      → 평가: [충족/미충족]
      → 근거:
      
      - BP2: 우성 변이와 trans에서 또는 병원성 변이와 cis에서 관찰
      → 평가: [충족/미충족]
      → 근거:
      
      - BP3: 알려진 기능이 없는 반복 영역의 in-frame indel
      → 평가: [충족/미충족]
      → 근거:
      
      - BP4: 다수의 전산 증거가 영향 없음을 시사
      → 평가: [충족/미충족]
      → 근거:
      
      - BP5: 대체 분자적 근거가 있는 증례에서 발견된 변이
      → 평가: [충족/미충족]
      → 근거:
      
      - BP6: 신뢰할 수 있는 출처에서 양성으로 분류
      → 평가: [충족/미충족]
      → 근거:
      
      - BP7: 스플라이스 영향 예측이 없는 동의 변이
      → 평가: [충족/미충족]
      → 근거: [SpliceAI, 보존성 고려]

3. 최종 분류:

   적용된 기준 요약:
   - 병원성: [충족된 모든 병원성 기준과 강도 나열]
   - 양성: [충족된 모든 양성 기준과 강도 나열]

   분류 규칙 (ACMG/AMP 2015):
   - 병원성 (Pathogenic): (i) 매우 강력 1개 + 강력 1개, 또는 (ii) 매우 강력 1개 + 중등도 ≥2개, 또는 (iii) 매우 강력 1개 + 중등도 1개 + 지지 1개, 또는 (iv) 강력 2개, 또는 (v) 강력 1개 + 중등도 ≥3개, 또는 (vi) 강력 1개 + 중등도 2개 + 지지 ≥2개
   - 병원성 가능 (Likely Pathogenic): (i) 매우 강력 1개 + 중등도 1개, 또는 (ii) 강력 1개 + 중등도 1-2개, 또는 (iii) 강력 1개 + 지지 ≥2개, 또는 (iv) 중등도 ≥3개, 또는 (v) 중등도 2개 + 지지 ≥2개, 또는 (vi) 중등도 1개 + 지지 ≥4개
   - 양성 (Benign): BA1 단독 또는 강력 양성 ≥2개
   - 양성 가능 (Likely Benign): 강력 양성 1개 + 지지 양성 1개 또는 지지 양성 ≥2개
   - 임상적 의미 불명 (VUS): 위 범주 기준 미충족

   최종 분류: [병원성/병원성 가능/VUS/양성 가능/양성]
   
   신뢰도: [높음/중등도/낮음]
   
   해석 요약: [간략한 임상적 해석]
   
   권고사항: [VUS인 경우 추가 증거 또는 검사 제안]

체계적 분석을 시작하세요:"""
        
        return self.chat(prompt, max_tokens=30000, temperature=0.2)



./anaconda3/envs/qwen-vl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print()
print("Qwen3-14B ACMG Variant Interpretation System")
print()
hf_token="hf_****************************"
# 모델 로드
model = Qwen3_14B(hf_token)



Qwen3-14B ACMG Variant Interpretation System

Qwen3-14B Loader

Model: Qwen/Qwen3-14B
Download size: 29.6GB
Expected memory: ~38GB during inference
Available VRAM: 48GB

GPUs available: 2
  GPU 0: NVIDIA GeForce RTX 3090 - 23.6GB
  GPU 1: NVIDIA GeForce RTX 3090 - 23.6GB

Cleaning GPU memory...

Step 1/2: Loading tokenizer...
  Tokenizer loaded

Step 2/2: Loading model (this takes 3-5 minutes)...



`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.12s/it]



Model loaded successfully!

GPU Memory Usage

GPU 0: NVIDIA GeForce RTX 3090
  Allocated:   12.52GB / 23.6GB ( 53.1%)
  Reserved:    12.53GB
  Free:        11.04GB

GPU 1: NVIDIA GeForce RTX 3090
  Allocated:   14.98GB / 23.6GB ( 63.6%)
  Reserved:    14.99GB
  Free:         8.58GB

Total:
  Allocated:   27.51GB / 48GB ( 57.3%)
  Available:   20.49GB

Layer distribution:
  0: 19 layers
  1: 25 layers




In [3]:
def parse_vep_text(vep_text):
    """
    VEP 텍스트 출력을 dictionary로 파싱하면서 빈 값 처리
    형식: Field\tValue
    """
    vep_dict = {}
    lines = vep_text.strip().split('\n')
    
    for line in lines:
        if '\t' in line:
            parts = line.split('\t')
            if len(parts) >= 2:
                key = parts[0].strip()
                value = parts[1].strip()
                
                # VEP의 빈 값 표시들을 None으로 통일
                if value in ['-', '.', '', 'NA', 'N/A']:
                    vep_dict[key] = None
                else:
                    vep_dict[key] = value
    
    return vep_dict


def extract_variant_info_from_vep(vep_data):
    """
    VEP 출력에서 ACMG 변이 해석에 필요한 정보만 추출
    """
    
    if isinstance(vep_data, str):
        vep_data = parse_vep_text(vep_data)  # ← 이미 빈 값이 None으로 처리됨
    
    # 헬퍼 함수들
    def safe_get(key, default='N/A'):
        """None을 기본값으로 변환"""
        value = vep_data.get(key)
        return default if value is None else value
    
    def safe_get_numeric(key, default=None):
        """숫자형 필드 안전하게 가져오기"""
        value = vep_data.get(key)
        if value is None:
            return default
        try:
            return float(value)
        except (ValueError, TypeError):
            return default
    
    variant_info = {
        # ================================================================
        # 1. 기본 변이 정보
        # ================================================================
        'gene': safe_get('SYMBOL'),
        'variant': safe_get('HGVSc'),
        'type': convert_consequence_to_type(safe_get('Consequence', '')),
        'cdna': safe_get('HGVSc'),
        'protein': safe_get('HGVSp'),
        'transcript': safe_get('Feature'),
        'genome_build': 'GRCh38',
        'chromosome': safe_get('Location', '').split(':')[0] if safe_get('Location') and ':' in safe_get('Location', '') else 'N/A',
        'position': safe_get('Location', '').split(':')[1].split('-')[0] if safe_get('Location') and ':' in safe_get('Location', '') else 'N/A',
        'ref_allele': safe_get('REF_ALLELE'),
        'alt_allele': safe_get('Allele'),
        
        # ================================================================
        # 2. 집단 빈도 데이터 (숫자형으로 변환)
        # ================================================================
        'gnomad_af': safe_get_numeric('gnomAD4.1_joint_AF'),
        'gnomad_ac': safe_get_numeric('gnomAD4.1_joint_AC'),
        'gnomad_an': safe_get_numeric('gnomAD4.1_joint_AN'),
        'gnomad_hom': safe_get_numeric('gnomAD4.1_joint_nhomalt'),
        'gnomad_popmax': safe_get_numeric('gnomAD4.1_joint_POPMAX_AF'),
        'gnomad_popmax_population': safe_get('gnomAD4.1_joint_POPMAX_POP'),
        
        # 집단별 빈도
        'gnomad_af_afr': safe_get_numeric('gnomAD4.1_joint_AFR_AF'),
        'gnomad_af_amr': safe_get_numeric('gnomAD4.1_joint_AMR_AF'),
        'gnomad_af_asj': safe_get_numeric('gnomAD4.1_joint_ASJ_AF'),
        'gnomad_af_eas': safe_get_numeric('gnomAD4.1_joint_EAS_AF'),
        'gnomad_af_fin': safe_get_numeric('gnomAD4.1_joint_FIN_AF'),
        'gnomad_af_nfe': safe_get_numeric('gnomAD4.1_joint_NFE_AF'),
        'gnomad_af_sas': safe_get_numeric('gnomAD4.1_joint_SAS_AF'),
        
        # ================================================================
        # 3. 데이터베이스 증거
        # ================================================================
        'clinvar': safe_get('clinvar_clnsig', 'Not found'),
        'clinvar_stars': safe_get('clinvar_review'),
        'clinvar_id': safe_get('clinvar_id'),
        'cosmic': extract_cosmic_id(safe_get('Existing_variation', '')),
        
        # ================================================================
        # 4. 전산 예측 도구 (숫자형)
        # ================================================================
        'revel': safe_get_numeric('REVEL_score'),
        'cadd': safe_get_numeric('CADD_phred') or safe_get_numeric('DANN_score'),
        'bayesdel': safe_get_numeric('BayesDel_addAF_score'),
        'metasvm': safe_get('MetaSVM_pred'),
        'metalr': safe_get('MetaLR_pred'),
        'metarnn': safe_get_numeric('MetaRNN_score'),
        
        # 미스센스 예측
        'sift': safe_get('SIFT'),
        'sift_score': extract_score_from_prediction(safe_get('SIFT', '')),
        'polyphen2_hvar': safe_get('PolyPhen'),
        'polyphen2_hvar_score': extract_score_from_prediction(safe_get('PolyPhen', '')),
        'mutation_taster': safe_get('MutationTaster_pred'),
        'mutation_assessor': safe_get('MutationAssessor_pred'),
        'provean': safe_get('PROVEAN_pred'),
        'primateai': safe_get_numeric('PrimateAI_score'),
        'vest4': None,
        'm_cap': safe_get_numeric('MPC_score'),
        
        # 스플라이싱 예측
        'spliceai_ds_ag': safe_get_numeric('SpliceAI_pred_DS_AG'),
        'spliceai_ds_al': safe_get_numeric('SpliceAI_pred_DS_AL'),
        'spliceai_ds_dg': safe_get_numeric('SpliceAI_pred_DS_DG'),
        'spliceai_ds_dl': safe_get_numeric('SpliceAI_pred_DS_DL'),
        'spliceai_max': calculate_spliceai_max(vep_data),
        
        # 보존성
        'phylop100way': safe_get_numeric('phyloP100way_vertebrate'),
        'phylop30way': safe_get_numeric('phyloP470way_mammalian'),
        'phastcons100way': safe_get_numeric('phastCons100way_vertebrate'),
        'phastcons30way': safe_get_numeric('phastCons470way_mammalian'),
        'gerp': safe_get_numeric('GERP++_RS'),
        
        # ================================================================
        # 5. 단백질/도메인 정보
        # ================================================================
        'protein_domain': safe_get('Interpro_domain'),
        'protein_position': safe_get('Protein_position'),
        'amino_acids': safe_get('Amino_acids'),
        'codons': safe_get('Codons'),
        'exon_number': safe_get('EXON', '').split('/')[0] if safe_get('EXON') and '/' in safe_get('EXON', '') else 'N/A',
        'total_exons': safe_get('EXON', '').split('/')[1] if safe_get('EXON') and '/' in safe_get('EXON', '') else 'N/A',
        
        # ================================================================
        # 6. 기타 정보
        # ================================================================
        'existing_variation': safe_get('Existing_variation'),
        'pubmed': safe_get('PUBMED'),
        'biotype': safe_get('BIOTYPE'),
        'canonical': safe_get('CANONICAL'),
        'impact': safe_get('IMPACT'),
        
        # AlphaMissense
        'alphamissense': safe_get_numeric('am_pathogenicity'),
        'alphamissense_class': safe_get('am_class'),
    }
    
    return variant_info


def calculate_spliceai_max(vep_data):
    """SpliceAI 최대값 계산 (안전하게)"""
    scores = []
    for key in ['SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 
                'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']:
        value = vep_data.get(key)
        if value is not None:
            try:
                scores.append(float(value))
            except (ValueError, TypeError):
                continue
    
    return max(scores) if scores else None


def extract_score_from_prediction(pred_string):
    """예측 문자열에서 점수 추출 (예: 'deleterious(0.01)' -> 0.01)"""
    if not pred_string or pred_string == 'N/A':
        return None
    
    if '(' in pred_string and ')' in pred_string:
        try:
            score = pred_string.split('(')[1].split(')')[0]
            return float(score)
        except (ValueError, IndexError):
            return None
    
    return None


def extract_cosmic_id(existing_var):
    """Existing_variation에서 COSMIC ID 추출"""
    if not existing_var or existing_var == 'N/A':
        return 'N/A'
    
    if 'COSV' in existing_var:
        parts = existing_var.split(',')
        for part in parts:
            if 'COSV' in part:
                return part.strip()
    
    return 'N/A'


def convert_consequence_to_type(consequence):
    """VEP consequence를 간단한 변이 타입으로 변환"""
    if not consequence:
        return 'N/A'
    
    consequence_map = {
        'missense_variant': 'missense',
        'synonymous_variant': 'synonymous',
        'stop_gained': 'nonsense',
        'frameshift_variant': 'frameshift',
        'inframe_insertion': 'in_frame_indel',
        'inframe_deletion': 'in_frame_indel',
        'splice_acceptor_variant': 'splice_site',
        'splice_donor_variant': 'splice_site',
        'splice_region_variant': 'splice_region',
        'start_lost': 'start_loss',
        'stop_lost': 'stop_loss',
    }
    
    for vep_term, simple_type in consequence_map.items():
        if vep_term in consequence:
            return simple_type
    
    return consequence

## 변이 해석 

In [4]:
import pandas as pd
import os

# 저장 디렉토리 생성
output_dir = "./instruct_results"
os.makedirs(output_dir, exist_ok=True)

vep_df = pd.read_csv("./variants.annot.txt", sep="\t")

for i in range(len(vep_df)):

    vep_var = extract_variant_info_from_vep(vep_df.iloc[i])
    
    print()
    print("="*70)
    print(f"TEST {i}: Variant Analysis")
    print("="*70)
    print()
    
    # 분석
    output = model.analyze_variant(vep_var)
    print(output)
    
    # Markdown 파일로 저장
    gene = vep_var.get('gene', 'Unknown')
    variant = vep_var.get('variant', 'Unknown').replace(':', '_').replace('>', '_')
    
    filename = f"{output_dir}/variant_{i:03d}_{gene}_{variant}.md"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"# Variant Analysis Report #{i}\n\n")
        f.write(f"**Gene:** {gene}\n\n")
        f.write(f"**Variant:** {vep_var.get('variant', 'N/A')}\n\n")
        f.write("---\n\n")
        f.write(output)
    
    print(f"✓ Saved to: {filename}")
    print()
    print("="*70)
    print(f"Test {i} completed!")
    print("="*70)


TEST 0: Variant Analysis

Generating response...

Generated 3499 tokens in 197.27s (17.7 tok/s)

<think>
Okay, let's tackle this variant interpretation step by step. The variant in question is VHL gene, c.74C>T (p.Pro25Leu). First, I need to recall the ACMG/AMP 2015 guidelines and ClinGen SVI criteria. 

Starting with the Pathogenic criteria. PVS1 is for loss-of-function variants in genes where LOF is a known disease mechanism. VHL is a tumor suppressor gene, and LOF variants are indeed pathogenic. However, this is a missense variant, not a LOF. So PVS1 is not applicable here. 

Next, PS1: same amino acid change as a known pathogenic variant. I don't see any information on ClinVar or HGMD indicating that this specific change is listed as pathogenic. ClinVar says Benign, but that might be from a different source. Need to check if there's a known pathogenic variant at Pro25. If not, PS1 isn't met.

PS2: De novo in a patient without family history. The data doesn't mention de novo status